In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import matplotlib.pyplot as plt

In [3]:
# Model / data parameters
num_classes = 10
input_shape = (32, 32, 3)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# reshape to write input shape
x_train = np.array([cv2.cvtColor(cv2.resize(x, input_shape[:2], interpolation = cv2.INTER_AREA),cv2.COLOR_GRAY2RGB) for x in x_train])
x_test = np.array([cv2.cvtColor(cv2.resize(x, input_shape[:2], interpolation = cv2.INTER_AREA),cv2.COLOR_GRAY2RGB) for x in x_test])
# Make sure images have shape (28, 28, 1)
# x_train = np.expand_dims(x_train, -1)
# x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 32, 32, 3)
60000 train samples
10000 test samples


In [4]:
base_model = keras.applications.MobileNet(
    input_shape=input_shape,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)
base_model.trainable = False
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 16, 16, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 16, 16, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 16, 16, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 16, 16, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 16, 16, 32)       128       
 ation)                                         

In [5]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [6]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 10s 6ms/step - loss: 0.3757 - accuracy: 0.8853 - val_loss: 0.0886 - val_accuracy: 0.9757
Epoch 2/15
422/422 [==============================] - 2s 5ms/step - loss: 0.1158 - accuracy: 0.9650 - val_loss: 0.0627 - val_accuracy: 0.9818
Epoch 3/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0868 - accuracy: 0.9732 - val_loss: 0.0459 - val_accuracy: 0.9873
Epoch 4/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0700 - accuracy: 0.9784 - val_loss: 0.0408 - val_accuracy: 0.9873
Epoch 5/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0605 - accuracy: 0.9813 - val_loss: 0.0361 - val_accuracy: 0.9898
Epoch 6/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0555 - accuracy: 0.9829 - val_loss: 0.0338 - val_accuracy: 0.9903
Epoch 7/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0503 - accuracy: 0.9843 - val_loss: 0.0318 - val_accuracy: 0.9918
Epoch

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.025982141494750977
Test accuracy: 0.9907000064849854
